# ED2 Jobs


## Background

The Ecosystem Demography Biosphere Model (ED2) is an integrated terrestrial biosphere model incorporating hydrology, land-surface biophysics, vegetation dynamics, and soil carbon and nitrogen biogeochemistry (Longo et al. 2019;Medvigy et al., 2009). Like its predecessor, ED (Moorcroft et al., 2001), ED2 uses a set of size- and age-structured partial differential equations that track the changing structure and composition of the plant canopy. With the ED2 model, in contrast to conventional biosphere models in which ecosystems with climatological grid cells are represented in a highly aggregated manner, the state of the aboveground ecosystem is described by the density of trees of different sizes and how this varies across horizontal space for a series of plant functional types. For more details, please go [here](https://github.com/EDmodel/*ED2*).

## Run ED2 jobs on HPC cluster

This notebook run ED2 jobs on HPC clusters. This currently works with servers that can be connected  via single-factor authentication. You can submit jobs for tonzi, harvest and umbs. There is a separate notebook for [santarem job.](https://colab.research.google.com/drive/1JAoSEL8g_nDsyqpXieHWDVYh2Vg_ufM0#scrollTo=3j6LbGTUR-XB)

## Prerequisites
The following modules are necessary to run this notebook. Use pip install

1. paramiko

In [ ]:
!pip install paramiko

In [ ]:
import getpass
import paramiko
import stat
import os
import time as timer
import subprocess

## Server Details



In [ ]:
hostname = 'cc-login.campuscluster.illinois.edu' # the hostname of the cluster you want to run the ed2 model
username = "ABC" # username on the cluster
password = None

## Job output details

In [ ]:
show_status = False # Set this to see ocurrent status of the job you submitted
show_output = False # Set this to see the output of the job

## Batch job details
Feel free to ignore the parameters if you wish to keep the default.

In [ ]:
# Batch job details
time = "00:15:00"                       # Job run time (hh:mm:ss)
nodes = 1                               # Number of nodes
ntasks_per_node = 16                    # Number of task (cores/ppn) per node
job_name = "ED2IN-umbs.bg"              # Name of batch job
partition = "secondary"                 # Partition (queue)
output = "openmp_" + job_name + ".o%j"  # Name of batch job output file
error = "openmp_" + job_name + ".e%j"   # Name of batch job error file
mail_user = "ABC@illinois.edu"        # Send email notifications
mail_type = "BEGIN,END"                 # Type of email notifications to send

## Path to input data for job

In [ ]:
# Path to various inputs
path_to_data = "${HOME}/ed-demo" #path to data for ED2 on cluster
path_to_singularity_image = "${HOME}/ed2-intel.sif" # path to singularity image of ED2 model on cluster
path_to_ED2IN_file = "ED2IN-umbs.bg" # path to ED2IN file on cluster, relative path to path_to_data is okay

In [ ]:
def submit_job(username):
    ssh_client = paramiko.SSHClient()
    ssh_client.load_system_host_keys()
    ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    try:
        ssh_client.connect(hostname, username=username, password=password, allow_agent=True)
    except:
        pass
    transport = ssh_client.get_transport()
    transport.auth_password(username, getpass.getpass('Enter {0} Logon password :'.format(hostname)))
    sftp_client = paramiko.SFTPClient.from_transport(transport)

    #create the bat file
    with open(job_name + ".sbatch", 'w') as f:
        f.writelines("#!/bin/bash\n")
        f.writelines("#SBATCH --time=" + str(time) + "\n")
        f.writelines("#SBATCH --ntasks-per-node=" + str(ntasks_per_node) + "\n")
        f.writelines("#SBATCH --job-name=" + job_name + "\n")
        f.writelines("#SBATCH --partition=" + partition + "\n")
        f.writelines("#SBATCH --output=" + output + "\n")
        f.writelines("#SBATCH --error=" + error + "\n")
        f.writelines("#SBATCH --mail-user=" + mail_user + "\n")
        f.writelines("#SBATCH --mail-type=" + mail_type + "\n")
        f.writelines("\n")
        f.writelines("module load singularity" + "\n")
        f.writelines("singularity exec --bind " + path_to_data + ":/data --no-home --pwd /data "
                       + path_to_singularity_image + " ed2 -f " + path_to_ED2IN_file)
    f.close()

    #transfer .bat file to cluster and run it
    sftp_client.put(job_name + ".sbatch", f"/home/{username}/" + job_name + ".sbatch")
    sftp_client.chmod(f"/home/{username}/" + job_name + ".sbatch", stat.S_IRWXU)
    _, stdo, stde = ssh_client.exec_command("sbatch " + job_name + ".sbatch")
    print(stde.read().decode())

    # Extract the job ID from the sbatch output
    result = stdo.read().decode()
    print(result)
    job_id = result.split()[3]

    # Show job status
    if show_status:
        # Check the job status periodically
        while True:
            #job_status = subprocess.run(f"squeue -u {username} -j {job_id}", shell=True, capture_output=True, text=True)
            _, stdo, stde = ssh_client.exec_command(f"squeue -u {username} -j {job_id}")
            job_status = stdo.read().decode()
            print(job_status)

            # Break the loop if the job is completed or failed
            if job_id not in job_status:
                break

            # Wait for a few seconds before checking again
            timer.sleep(10)
    if show_output:
        print("Output")
        # View output
        _, stdo, stde = ssh_client.exec_command(f"cat /home/{username}/openmp_{job_name}.o{job_id}")
        print(stdo.read().decode())

        print("Error")
        # View error
        _, stdo, stde = ssh_client.exec_command(f"cat /home/{username}/openmp_{job_name}.e{job_id}")
        print(stdo.read().decode())


    sftp_client.close()
    ssh_client.close()
    transport.close()
submit_job(username)

Enter cc-login.campuscluster.illinois.edu Logon password :··········

Submitted batch job 9903665

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           9903665 secondary ED2IN-um    ddey2 PD       0:00      1 (None)

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           9903665 secondary ED2IN-um    ddey2  R       0:12      1 ccc0162

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           9903665 secondary ED2IN-um    ddey2  R       0:22      1 ccc0162

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           9903665 secondary ED2IN-um    ddey2  R       0:33      1 ccc0162

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           9903665 secondary ED2IN-um    ddey2  R       0:43      1 ccc0162

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
       